In [1]:
from parser import Parser
from tokenizer import Tokenize
from generator import Generate


In [2]:
code = 'g := Lx.(x + 2)'

tokens = Tokenize(code)
parser = Parser(tokens)
ast = parser.parse()
py_code = Generate(ast)

print(py_code)  # Output: f = lambda x: x + 1

g = lambda x: x + 2


In [3]:
code = 'f := Lx.(x + 1)\ng := Lx.(x + 2)'
def interpret(code):
    
    for e in code.split('\n'):
        tokens = Tokenize(e)
        parser = Parser(tokens)
        ast = parser.parse()
        py_code = Generate(ast)
        
        print(py_code) 

interpret(code)

f = lambda x: x + 1
g = lambda x: x + 2


In [4]:
from interpreter import interpret

interpret(code)

'f = lambda x: x + 1\ng = lambda x: x + 2\n'

In [5]:
import subprocess
import sys
import threading

class PythonREPL:
    def __init__(self, python_cmd=None, prompt=">>> "):
        """
        Spawn a fresh, un-buffered, interactive Python subprocess
        and sync up until its first prompt.
        """
        python_cmd = python_cmd or [sys.executable, "-u", "-i"]
        self.prompt = prompt
        self.proc = subprocess.Popen(
            python_cmd,
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True,
            bufsize=1,            # line-buffered
        )
        # drain the initial banner up to the first '>>> '
        self._read_until(self.prompt)

    def _read_until(self, marker):
        """
        Read from self.proc.stdout until the output ends with `marker`.
        Returns everything read (including the marker).
        """
        buf = ""
        while not buf.endswith(marker):
            chunk = self.proc.stdout.read(1)
            if chunk == "":
                # EOF
                break
            buf += chunk
        return buf

    def run(self, code):
        """
        Send a chunk of code (one line or many), then read back
        everything up to the next prompt.
        Returns the full raw text (including the trailing prompt).
        """
        if not code.endswith("\n"):
            code = code + "\n"
        self.proc.stdin.write(code)
        self.proc.stdin.flush()
        return self._read_until(self.prompt)

    def close(self):
        """Terminate the subprocess."""
        self.proc.terminate()
        self.proc.wait()


# === example usage ===
repl = PythonREPL()
out1 = repl.run("x = 123")               # no visible output, just sync
out2 = repl.run("print(x * 2)")
print("RAW output from subprocess:\n", out2)
repl.close()


RAW output from subprocess:
 246
>>> 
